In [1]:
# Import necessary packages
import json  # For handling JSON data
from openai import OpenAI  # For interacting with OpenAI API
import os  # For interacting with the operating system, such as file paths
import re  # For regular expressions, useful for pattern matching in strings
import pandas as pd

In [76]:
# Function to convert letter to number
def letter_to_number(letter):
    return str(ord(letter) - ord('A') + 10)

# Function to parse matrix content
def parse_matrix(matrix_content):
    data = []
    headers = []
    lines = matrix_content.strip().split('\n')
    for i in range(0, len(lines), 2):
        taxa = lines[i].strip().strip("'")
        traits = lines[i + 1].strip()
        species_traits = []
        j = 0
        while j < len(traits):
            if traits[j] == '(':
                j += 1
                states = ''
                while traits[j] != ')':
                    if traits[j].isalpha():
                        states += letter_to_number(traits[j])
                    else:
                        states += traits[j]
                    j += 1
                species_traits.append(','.join(states))
            elif traits[j] == '?':
                species_traits.append('Missing')
            elif traits[j] == '-':
                species_traits.append('Not Applicable')
            elif traits[j].isalpha():
                species_traits.append(letter_to_number(traits[j]))
            else:
                species_traits.append(traits[j])
            j += 1
        data.append([taxa] + species_traits)
    max_traits = max(len(row) - 1 for row in data)
    headers = ['taxa'] + [f'Character{i + 1}' for i in range(max_traits)]
    try:
        df = pd.DataFrame(data, columns=headers)
        return df
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return None

# Function to convert NEXUS to CSV
def convert_nexus_to_csv(file_path, output_path):
    try:
        encodings = ['utf-8', 'gbk', 'latin1']  # List of encodings to try
        for encoding in encodings:
            try:
                with open(file_path, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"Successfully read file with encoding: {encoding}")
                break
            except UnicodeDecodeError:
                print(f"Failed to read file with encoding: {encoding}")
                continue
        else:
            raise ValueError("Failed to read file with all attempted encodings.")
        
        matrix_content = re.search(r'MATRIX\s*(.*?)\s*;', content, re.DOTALL).group(1).strip()
        df = parse_matrix(matrix_content)
        df.to_csv(output_path, index=False)
        return df
    except FileNotFoundError:
        print(f"File {file_path} not found.")
    except Exception as e:
        print(f"Appear error：{e}")

# Function to build knowledge graph from CSV
def build_knowledge_graph(matrix):
    knowledge_graph = {}
    for _, row in matrix.iterrows():
        taxa = row.iloc[0]
        characteristics = {}
        for col in matrix.columns[1:]:
            state = row[col]
            if isinstance(state, str) and ',' in state:
                state = state.replace(',', ' and ')
            characteristics[col] = str(state)
        knowledge_graph[taxa] = {'Characteristics': characteristics}
    return knowledge_graph

# Function to save knowledge graph as JSON
def save_knowledge_graph_as_json(knowledge_graph, file_path):
    with open(file_path, 'w') as f:
        json.dump(knowledge_graph, f, indent=4)

# Main function to combine all steps
def nexus_to_knowledge_graph(nexus_file_path, csv_output_path, json_output_path):
    # Step 1: Convert NEXUS to CSV
    df = convert_nexus_to_csv(nexus_file_path, csv_output_path)
    
    if df is not None:
        # Step 2: Build the knowledge graph from the DataFrame
        knowledge_graph = build_knowledge_graph(df)
        
        # Step 3: Save the knowledge graph as a JSON file
        save_knowledge_graph_as_json(knowledge_graph, json_output_path)
        
        # Optional: Print the JSON structure for verification
        knowledge_graph_json = json.dumps(knowledge_graph, indent=4)
        # print(knowledge_graph_json)
        
        # Return the knowledge graph for further use
        return knowledge_graph
    else:
        print("Failed to create the DataFrame from NEXUS file.")
        return None

# Example Usage
nexus_file_path = "D:/桌面/taxonomy_primary_result/The_GPT-4_result/Dataset_10 (Revision of the Oriental genera of Agathidina) 18/Information gain methods/nexdata"
csv_output_path = "D:/桌面/process_data_2.csv"
json_output_path = "D:/桌面/knowledge_graph.json"
# Process the file and get the knowledge graph
knowledge_graph = nexus_to_knowledge_graph(nexus_file_path, csv_output_path, json_output_path)


Successfully read file with encoding: utf-8


In [5]:
# print(knowledge_graph)

{'Agathis': {'Characteristics': {'Character1': '4', 'Character2': '1 and 2', 'Character3': '2', 'Character4': '2', 'Character5': '2', 'Character6': '1', 'Character7': '2', 'Character8': '1', 'Character9': '1', 'Character10': '2', 'Character11': '1', 'Character12': '1', 'Character13': '2', 'Character14': '2', 'Character15': '3', 'Character16': '2', 'Character17': '2'}}, 'Lytopylus': {'Characteristics': {'Character1': '4', 'Character2': '2', 'Character3': '2', 'Character4': '1 and 2', 'Character5': '2', 'Character6': '2', 'Character7': '2', 'Character8': '1', 'Character9': '1', 'Character10': '2', 'Character11': '1', 'Character12': '1', 'Character13': '2', 'Character14': '1', 'Character15': '3', 'Character16': '2', 'Character17': '2'}}, 'Braunsia': {'Characteristics': {'Character1': '4', 'Character2': '1 and 2', 'Character3': '2', 'Character4': '1', 'Character5': '2', 'Character6': '2', 'Character7': '2', 'Character8': '1', 'Character9': '2', 'Character10': '2', 'Character11': '1', 'Char

In [6]:
# Input the API key and morphological matrix
# Initialize the OpenAI client with the API key from environment variables
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# This code sets up the prompt for the API input using the client.chat.completions.create interface
# to conduct multi-turn conversations. By assigning different roles in the conversation (user, assistant, system),
# all input information is fully conveyed to the API model.
messages_initial = [
    # Set the system role to focus on taxonomy tasks.
    # Emphasize the system's understanding of morphological matrices, information gain,
    # and the construction of classification keys in the system setup.
    {"role": "system",
     "content":
         """You are a helpful taxonomist assistant.
         You are skilled at calculating the correct information gain to choose the character that best divides species into even groups based on their states.
         Based on the selected character, classify the species into different groups according to their states.
         For each group with more than two species, continue selecting characters to further classify this group until each group only has one species.
         After multiple classifications, determine the final classification levels and record each classifying character and its state. 
         Finally, generate a taxonomic key.
         You need to strictly ensure that you categorize all species when selecting the initial character, and that you don't ignore any of the species.
         Please format the classification result as follows:
        ```
        {
            "Character": "Character1",
            "States": {
                "State 1": ["species1", "species2", ...],
                "State 2": ["species3", "species4", ...],
                "State ...": ["species5", "species6", ...]
            }
        }
        ```
        Ensure that the response follows this format exactly.
        Additionally,exactly ensure that all species are included in the initial classification result.
         """},

    # Input the main task request, construct the classification key, and emphasize the details needed for classification.
    # This includes: the main objective, information gain calculation, ignoring invalid states, clarifying multi-character states,
    # guiding correct selection based on the significance of information gain, and standardizing the final output format for subsequent extraction of API results.
    {"role": "system",
     "content": """
                Generate the taxonomic key based on the provided morphological matrix. The matrix includes all species and their different states for each character.
                The process involves selecting a character to classify the species into groups. Repeat this classification within each subgroup until each group contains only one species.
                Information gain measures how much the uncertainty in the dataset is reduced after using a character for classification. It helps in selecting characters that minimize the entropy of the subset after classification, leading to better classification results.
                Please select the initial classification character for all species based on the morphological matrix and information gain methods.
                In the morphological matrix, 'Missing' and 'Not applicable' are invalid states. If a character has invalid states for the group being classified, it should be ignored.
                States are represented by numbers, and like the '1 and 2' means multiple states should be treated as a single state type.(such as '3' and '2 and 3'  is the different state, these are two separate states, when i choose character to based on different state to distinguish the species)
                You need to calculate the information gain for each character, and choose the highest information gain result, The higher the Information Gain result, the greater the contribution of the feature to the classification.
                Now I will show you the morphological matrix. Please provide only the initial classification character and the categorization of species based on its state, and you should singly show this called as # initial character classify result #
            """},

    # Use the assistant to summarize and refine the prompt content for the API.
    # Through the conversation with the assistant, deepen the API understanding of the content to some extent,
    # control the API response results, and standardize the output format of the API response.
    {"role": "assistant",
     "content": """
                Understood. I will generate the taxonomic key based on the provided morphological matrix. Here is a summary of the steps I will follow:
                1. The matrix includes all species and their different states for each character.
                2. I will select a character to classify the species into groups and repeat this classification within each subgroup until each group contains only one species, and i'll not ignore any species.
                3. I will use information gain to measure how much the uncertainty in the dataset is reduced after using a feature for classification. This helps in selecting features that minimize the entropy of the subset after classification, leading to better classification results.
                4. I will select the initial classification character for all species based on the morphological matrix and information gain methods.
                5. In the morphological matrix, 'Missing' and 'Not applicable' are considered invalid states. If a character has invalid states for the group being classified, it will be ignored.
                6. States are represented by numbers. For example, '1 and 2' means multiple states should be treated as a single state type. (such as '1' and '1 and 2'  is the different state, these are two separate states, when i choose character to based on different state to distinguish the species)
                7. I will use information gain to calculate all character and choose the highest information gain result, The higher the Information Gain result, the greater the contribution of the feature to the classification.so i need to make sure the result is Average classification
                8. The final result will provide only the initial classification character and the categorization of species based on its state.
                9. I will use all the species in their entirety, strictly making sure to categorize all of them! (need to make sure contain all species)
                10. Don't need to show how i calculate, only need to show the final result, and please show the final result in #initail character classify result# block, Don't have errors where the state and species don't match
                Please provide the morphological matrix data so that I can proceed with the initial classification.
            """},

    # Input the corresponding morphological matrix information for analysis.
    {"role": "user",
     "content": f"Here is morphological matrix:{knowledge_graph}"}
]

# Set various parameters to control the API response. 
# Setting the temperature to 0 and limiting max_tokens to save costs and avoid long, redundant outputs.
initial_character_info = client.chat.completions.create(
    model="gpt-4o",
    messages=messages_initial ,
    stop=None,
    max_tokens=1000,
    temperature=0,
    n=1
)

# Store the API call response results as a file. 
# (For subsequent distributed API call loops, consider storing in environment variables for continuous calls and modifications).
initial_response = initial_character_info.choices[0].message.content

# If used as a whole pipeline to transfer the results, ignore this print. 
# However, for debugging, you can use this print statement to check the response.
print(initial_response)


# Initial Character Classify Result #
```
{
    "Character": "Character1",
    "States": {
        "1": ["Biroia", "Coccygidium", "Disophrys", "Cremnoptoides"],
        "2": ["Troticus", "Hypsostypos", "Cremnops"],
        "3": ["Gyrochus"],
        "4": ["Agathis", "Lytopylus", "Braunsia", "Camptothlipsis", "Therophilus", "Bassus", "Earinus", "Amputostypos", "Euagathis", "Aneurobracon"]
    }
}
```


In [78]:
def parse_classification_result(result_text):
    classification = {"Character": None, "States": {}}
    try:
        # Attempt to match the Character
        character_match = re.search(r'"Character": "([^"]+)"', result_text)
        if character_match:
            classification["Character"] = character_match.group(1)
        else:
            raise ValueError("Character not found in the result text.")

        # Attempt to match each State and the corresponding species
        state_sections = re.findall(r'"(\d+|[^"]+)":\s*\[(.*?)\]', result_text)
        if not state_sections:
            raise ValueError("No states found in the result text.")

        for state, species_block in state_sections:
            species_list = re.findall(r'"([^"]+)"', species_block)
            if not species_list:
                raise ValueError(f"No species found for state {state}.")
            classification["States"][state] = species_list

    except Exception as e:
        print(f"Error parsing classification result: {e}")
        # Decide whether to return an empty classification or raise an exception when an error occurs
        raise e  # Or return classification

    return classification
print(type(initial_response))
parsed_initial_classification = parse_classification_result(initial_response)
print(parsed_initial_classification)

# Function to generate groups from the classification result
def generate_groups_from_classification(classification_result):
    """
    Generate groups from classification result.
    
    :param classification_result: Dictionary containing the classification result
    :return: List of tuples, where each tuple contains a state and a list of species
    """
    groups = []
    for state, species_list in classification_result["States"].items():
        groups.append((state, species_list))
    return groups

# Generate groups from the parsed initial classification
groups = generate_groups_from_classification(parsed_initial_classification)
print(groups)
print(groups[0])
print(groups[1])
print(groups[2])


<class 'str'>
{'Character': 'Character1', 'States': {'1': ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides'], '2': ['Troticus', 'Hypsostypos', 'Cremnops'], '3': ['Gyrochus'], '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']}}
[('1', ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides']), ('2', ['Troticus', 'Hypsostypos', 'Cremnops']), ('3', ['Gyrochus']), ('4', ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon'])]
('1', ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides'])
('2', ['Troticus', 'Hypsostypos', 'Cremnops'])
('3', ['Gyrochus'])


In [79]:
# API call function for continued grouping for each subgroup
def classify_group(group_species):
    # Create a sub-matrix for the group of species
    group_matrix = {species: knowledge_graph[species] for species in group_species}
    group_matrix_str = json.dumps(group_matrix, ensure_ascii=False)
    
    # Define the messages for the API call
    messages_secondary = [
        {"role": "system",
         "content":
             """
             You are a helpful taxonomist assistant.
             You are skilled at calculating the correct information gain to choose the character that best divides species into even groups based on their states.
             Based on the selected character, classify the species into different groups according to their states.
             For each group with more than two species, continue selecting characters to further classify this group until each group only has one species.
             After multiple classifications, determine the final classification levels and record each classifying character and its state.
             Finally, generate a taxonomic key.
             """},
        {"role": "system",
         "content":
             """
             Generate the nested taxonomic key based on the provided morphological matrix.
             The process involves selecting a character to classify the species into groups. Repeat this classification within each subgroup until each group contains only one species.
             Information gain measures how much the uncertainty in the dataset is reduced after using a character for classification. It helps in selecting characters that minimize the entropy of the subset after classification, leading to better classification results.
             Please select the classification character for these group's species based on the morphological matrix and information gain methods.
             In the morphological matrix, 'Missing' and 'Not applicable' are invalid states. If a character has invalid states for the group being classified, it should be ignored.
             States are represented by numbers. For example, '1 and 2' means multiple states should be treated as a single state type and this multi-state characterization should not be confused with the single states within it (the state of '3' and '2 and 3' is different state, when you choose the character to based on the state to distinguish need to careful handle). The initial character should have no more than three state types.
             You need to calculate the information gain for each character and choose the highest information gain result. The higher the information gain result, the greater the contribution of the feature to the classification.
             After selecting the initial classification character and categorizing the species based on its state, repeat the process within each subgroup. For each subgroup, select the character with the highest information gain to further classify the species. Continue this process recursively until each group contains only one species.
             Now I will show you the morphological matrix. Please provide the classification character and the categorization of species based on its state. Then, continue to classify each subgroup recursively, showing the chosen character and categorization for each subgroup. Please present the result in a structured format, with each step clearly labeled.
             Please don't show how you analyze and calculate, please show me the final result.
             """},
        {"role": "assistant",
         "content":
             """
             Understood. I will generate the nested taxonomic key based on the provided morphological matrix. Here is a summary of the steps I will follow:
             1. The matrix includes all species and their different states for each character.
             2. I will select a character to classify the species into groups and repeat this classification within each subgroup until each group contains only one species.
             3. I will use information gain to measure how much the uncertainty in the dataset is reduced after using a feature for classification. This helps in selecting features that minimize the entropy of the subset after classification, leading to better classification results.
             4. I will select the classification character for the group's species based on the morphological matrix and information gain methods.
             5. In the morphological matrix, 'Missing' and 'Not applicable' are considered invalid states. If a character has invalid states for the group being classified, it will be ignored.
             6. States are represented by numbers. For example, '2 and 3' means multiple states should be treated as a single state type, and this multi-state characterization should not be confused with the individual states (like '2', '3') within it (such as '3' and '2 and 3' are different states, these are two separate states, when I choose a character based on different states to distinguish the species). The classification character should have no more than three state types.
             7. I will use information gain to calculate all characters and choose the highest information gain result. The higher the information gain result, the greater the contribution of the feature to the classification.
             8. The final result will provide only the initial classification character and the categorization of species based on its state.
             9. Don't need to show how the process about choosing, only need to show the final result as a nested structure, and I will store the result in #character classify result# block.
             Please provide the group morphological matrix data so that I can proceed with the classification.
             """},
        {"role": "user", "content": f"Here is the group information need to be classify and include the morphological matrix {group_matrix_str}"}
    ]
    
    # Make the API call to classify the group
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_secondary,
        stop=None,
        temperature=0,
        max_tokens=1000,
        n=1
    )
    result_secondary = response.choices[0].message.content
    # print(f"API response for group {group_species}: {result}")
    
    # Define messages for formatting the response to JSON
    messages_JSON = [
        {"role": "system",
         "content":
             """
             You are a helpful JSON format converter.
             You can express the nested structure as a JSON result based on the corresponding content.
             """},
        {"role": "system",
         "content":
             """
             Please format the classification result as follows:
             ```
             # Final taxonomic key result JSON format #
             {
                 "Character": "CharacterX",
                 "States": {
                     "1": ["speciesA"],
                     "2": {
                         "Character": "CharacterY",
                         "States": {
                             "1": ["speciesB"],
                             "2": ["speciesC"]
                         }
                     }
                 }
             }
             ```
             Ensure that the response follows this format exactly.
             """},
        {"role": "assistant",
         "content":
             """
             Understood. I'll convert the nested structure you gave me into JSON format and store it in # final result #.
             Please provide what you need to convert the format.
             """},
        {"role": "user", "content": f"Here are the taxonomic results for the nested schema representation {result_secondary}"}
    ]
    
    # Make the API call to format the response as JSON
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_JSON,
        stop=None,
        temperature=0,
        max_tokens=1500,
        n=1
    )
    json_result = response.choices[0].message.content
    print(json_result)
    return json_result


In [80]:
# Function to clean and extract JSON string
def extract_json_string(json_string):
    # Find the positions of the start and end of the JSON object
    start = json_string.find('{')
    end = json_string.rfind('}') + 1
    
    # If both start and end positions are valid, extract and return the JSON string
    if start != -1 and end != -1:
        cleaned_string = json_string[start:end]
        return cleaned_string.strip()
    
    # If positions are not valid, return an empty string
    return ""


In [81]:
def recursive_classification(groups, final_classification, classification_results, depth=0, max_depth=10):
    """
    Recursive classification function to process groups and store results.
    :param groups: Groups to be processed
    :param final_classification: Final classification result
    :param classification_results: Classification results
    :param depth: Current recursion depth
    :param max_depth: Maximum recursion depth
    :return: Final classification result
    """
    # Continue looping while the groups list is not empty
    # Initialize state and current_group for error handling
    state, current_group = None, []
    while groups:
        try:
            # Pop the first group from the list, getting the state and current group of species
            state, current_group = groups.pop(0)
            print(f"Processing group with state: {state}, species: {current_group}, at depth: {depth}")

            # If the current group has only one species, add it to the final classification
            if len(current_group) == 1:
                final_classification[current_group[0]] = current_group
            # If the current recursion depth has reached the maximum depth, stop further classification
            elif depth >= max_depth:
                print(f"Reached max depth {max_depth}. Stopping further classification for group: {current_group}")
                final_classification[state] = current_group
            else:
                # Call the classify_group function to classify the current group
                classification_result = classify_group(current_group)
                # Clean the API classification result to extract the JSON string
                cleaned_classification_result = extract_json_string(classification_result)  
                # Store the classification result in classification_results
                classification_results[state] = cleaned_classification_result

                # Parse the classification result, create new subgroups, and add them to groups for further classification
                parsed_result = parse_classification_result(classification_result)
                new_groups = generate_groups_from_classification(parsed_result)

                # Recursively call itself to process new subgroups, increasing the recursion depth
                recursive_classification(new_groups, final_classification, classification_results, depth + 1, max_depth)

        except Exception as e:
            # Catch exceptions and print error messages
            print(f"Error processing group with state: {state}, species: {current_group}, at depth: {depth}")
            print(f"Exception: {e}")
            raise e

    return final_classification

In [82]:
# Assume the variables have been initialized
max_depth =  5  # Can be adjusted based on the hierarchical structure of input data and application requirements
# here is the initial character level is about species number need to classify

# Dictionary to store the final classification where each species is classified individually
final_classification = {}

# Dictionary to store the API classification results for each state
classification_results = {}

# Print the initial state of groups and dictionaries for debugging purposes
print("Initial groups:", groups)
print("Initial final_classification:", final_classification)
print("Initial classification_results:", classification_results)

# Call the recursive_classification function to process the groups and store the results
final_classification = recursive_classification(groups, final_classification, classification_results, depth=0, max_depth=max_depth)

# Print the final classification results
print("Final Classification:")
print(json.dumps(final_classification, indent=2, ensure_ascii=False))

# Print the classification results from the API calls
print("\nClassification Results:")
print(classification_results)


Initial groups: [('1', ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides']), ('2', ['Troticus', 'Hypsostypos', 'Cremnops']), ('3', ['Gyrochus']), ('4', ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon'])]
Initial final_classification: {}
Initial classification_results: {}
Processing group with state: 1, species: ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides'], at depth: 0
Here is the nested taxonomic key in JSON format:

```json
{
    "Character": "Character5",
    "States": {
        "1": ["Coccygidium"],
        "2": {
            "Character": "Character13",
            "States": {
                "1": ["Cremnoptoides"],
                "2": {
                    "Character": "Character10",
                    "States": {
                        "1": ["Disophrys"],
                        "2": ["Biroia"]
                    }
                }
            }
        }
    }
}
```
Processin

In [83]:
print(classification_results)
groups = generate_groups_from_classification(parsed_initial_classification)
print(groups)
print(classification_results)

{'1': '{\n    "Character": "Character5",\n    "States": {\n        "1": ["Coccygidium"],\n        "2": {\n            "Character": "Character13",\n            "States": {\n                "1": ["Cremnoptoides"],\n                "2": {\n                    "Character": "Character10",\n                    "States": {\n                        "1": ["Disophrys"],\n                        "2": ["Biroia"]\n                    }\n                }\n            }\n        }\n    }\n}', '2': '{\n    "Character": "Character 1",\n    "States": {\n        "1 and 2": ["Cremnops"],\n        "2": {\n            "Character": "Character 9",\n            "States": {\n                "1": ["Hypsostypos"],\n                "2": ["Troticus"]\n            }\n        }\n    }\n}', '4': '{\n    "Character": "Character2",\n    "States": {\n        "1 and 2": {\n            "Character": "Character3",\n            "States": {\n                "2": {\n                    "Character": "Character4",\n             

In [68]:
def extract_paths(node, path=None):
    if path is None:
        path = {}

    if 'Character' in node and 'States' in node:
        current_character = node['Character'].replace(" ", "").strip()
        for state, value in node['States'].items():
            new_path = path.copy()
            new_path[current_character] = state
            if isinstance(value, dict):
                yield from extract_paths(value, new_path)
            else:
                for species in value:
                    yield species, new_path

# 处理每个分类结果并提取路径
final_results = {}

for key, json_str in classification_results.items():
    classification_data = json.loads(json_str)
    species_paths = list(extract_paths(classification_data))

    formatted_results = {}
    for species, path in species_paths:
        formatted_results[species] = {"Characteristics": path}
    
    final_results[key] = formatted_results
    

In [69]:
def check_state_match(state, correct_state):
    if correct_state is None:
        return False
    if " and " in correct_state:
        correct_states = correct_state.split(" and ")
        return all(sub_state in correct_states for sub_state in state.split(" and "))
    return state == correct_state

# Validate classification results and log errors
def validate_results(final_results, knowledge_graph):
    errors = []
    for key, results in final_results.items():
        for species, data in results.items():
            if species in knowledge_graph:
                mismatch = False
                incorrect_character_states = {}
                for character, state in data["Characteristics"].items():
                    character = character.replace(" ", "").strip()
                    correct_state = knowledge_graph[species]["Characteristics"].get(character)
                    if correct_state is None or not check_state_match(state, correct_state):
                        mismatch = True
                        incorrect_character_states[character] = {"error_state": state, "correct_state": correct_state}
                if mismatch:
                    errors.append({
                        "species": species,
                        "key": key,
                        "error": "Mismatch",
                        "error_result": incorrect_character_states,
                        "correct_result": {character: knowledge_graph[species]["Characteristics"].get(character) for character in incorrect_character_states}
                    })
            else:
                errors.append({
                    "species": species,
                    "key": key,
                    "error": "Species not found in knowledge graph",
                    "error_result": data["Characteristics"]
                })
    return errors

In [70]:
def get_species_list_for_state(groups, key):
    species_list = []
    for state, species in groups:
        if state == key:
            species_list = species
            break
    if not species_list:
        print(f"Key {key} not found in groups")
    else:
        print(f"Processing species list for state '{key}': {species_list}")
    return species_list

In [71]:
def correct_classification(errors, classification_results, knowledge_graph):
    for error in errors:
        key = error['key']
        
        species_list = get_species_list_for_state(groups, key)
        if not species_list:
            continue
        
        group_matrix = {s: knowledge_graph[s] for s in species_list}
        group_matrix_str = json.dumps(group_matrix, ensure_ascii=False)  
        
        messages2 = [
            {"role": "system",
             "content": """
             You are a helpful taxonomist assistant.
             You are skilled at calculating the correct information gain to choose the character that best divides species into even groups based on their states.
             Based on the selected character, classify the species into different groups according to their states.
             For each group with more than two species, continue selecting characters to further classify this group until each group only has one species.
             After multiple classifications, determine the final classification levels and record each classifying character and its state.
             Finally, generate a taxonomic key.
             You are able to avoid the same error in your results based on the corrected results previously passed to you
             ***IMPORTANT: Ensure that each group contains only one species in the final classification result.***
             """},
            {"role": "system",
             "content": """
             Generate the nested taxonomic key based on the provided morphological matrix.
             The process involves selecting a character to classify the species into groups. Repeat this classification within each subgroup until each group contains only one species.
             Information gain measures how much the uncertainty in the dataset is reduced after using a character for classification. It helps in selecting characters that minimize the entropy of the subset after classification, leading to better classification results.
             Please select the classification character for these group's species based on the morphological matrix and information gain methods.
             In the morphological matrix, 'Missing' and 'Not applicable' are invalid states. If a character has invalid states for the group being classified, it should be ignored.
             States are represented by numbers. For example, '1 and 2' means multiple states should be treated as a single state type and this multi-state characterization should not be confused with the single states within it (the state of '3' and '2 and 3' is different state, when you choose the character to based on the state to distinguish need to careful handle). The initial character should have no more than three state types.
             You need to calculate the information gain for each character and choose the highest information gain result. The higher the information gain result, the greater the contribution of the feature to the classification.
             After selecting the initial classification character and categorizing the species based on its state, repeat the process within each subgroup. For each subgroup, select the character with the highest information gain to further classify the species. Continue this process recursively until each group contains only one species.
             Now I will show you the morphological matrix. Please provide the classification character and the categorization of species based on its state. Then, continue to classify each subgroup recursively, showing the chosen character and categorization for each subgroup. Please present the result in a structured format, with each step clearly labeled.
             Please don't show how you analyze and calculate, please show me the final result.
             """},
            {"role": "user", "content": f"""
            This is the result of the error you generated in the previous API call. 
            In this file I have provided you with the CORRECT result. 
            Please strictly adhere to the use of the correct species feature status message!{error}
            """},
            {"role": "assistant", "content": f"""
            I will strictly use the correct species feature state information for evaluation, 
            while I will avoid using these incorrect feature state information that appeared previously in the classification results
            """},
            {"role": "user", "content": f"Here is the group information need to be classify and include the morphological matrix {group_matrix_str}"}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages2,
            stop=None,
            temperature=0,
            max_tokens=1000,
            n=1
        )
        corrected_result = response.choices[0].message.content
        
        messages_JSON = [
        {"role": "system",
         "content":
             """
             You are a helpful JSON format converter.
             You can express the nested structure as a JSON result based on the corresponding content.
             """},
        {"role": "system",
         "content":
             """
             Please format the classification result as follows:
             ```
             # Final taxonomic key result JSON format #
             {
                 "Character": "CharacterX",
                 "States": {
                     "1": ["speciesA"],
                     "2": {
                         "Character": "CharacterY",
                         "States": {
                             "1": ["speciesB"],
                             "2": ["speciesC"]
                         }
                     }
                 }
             }
             ```
             Ensure that the response follows this format exactly.
             """},
        {"role": "assistant",
         "content":
             """
             Understood. I'll convert the nested structure you gave me into JSON format and store it in # final result #.
             Please provide what you need to convert the format.
             """},
        {"role": "user", "content": f"Here are the taxonomic results for the nested schema representation {corrected_result}"}
        ]
    
        # Make the API call to format the response as JSON
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages_JSON,
            stop=None,
            temperature=0,
            max_tokens=1500,
            n=1
        )
        json_result = response.choices[0].message.content
        json_cleaned_result = extract_json_string(json_result)
        print(json_cleaned_result)
        classification_results[key] = json_cleaned_result
        return classification_results

In [72]:
print(classification_results)

{'1': '{\n    "Character": "Character5",\n    "States": {\n        "1": ["Coccygidium"],\n        "2": {\n            "Character": "Character6",\n            "States": {\n                "1": ["Biroia"],\n                "1 and 2": {\n                    "Character": "Character13",\n                    "States": {\n                        "1": ["Cremnoptoides"],\n                        "2": ["Disophrys"]\n                    }\n                }\n            }\n        }\n    }\n}', '2': '{\n    "Character": "Character1",\n    "States": {\n        "2": {\n            "Character": "Character9",\n            "States": {\n                "2": ["Troticus"],\n                "1": ["Hypsostypos"]\n            }\n        },\n        "1 and 2": ["Cremnops"]\n    }\n}', '4': '{\n    "Character": "Character2",\n    "States": {\n        "1 and 2": {\n            "Character": "Character3",\n            "States": {\n                "2": {\n                    "Character": "Character4",\n          

In [84]:
# Cycle checks and corrections
errors = validate_results(final_results, knowledge_graph)
# Purpose: Enter a loop until all errors have been fixed.
# Function: Executes the code inside the loop when the errors list is not empty.
while errors:
    # Fix current categorization errors. (based on the API )
    classification_results = correct_classification(errors, classification_results, knowledge_graph)
    # Purpose: To reset the final_results dictionary to store the corrected categorization results.
    final_results = {}
    # Iterate over the corrected classification results and extract species classification paths.
    for key, json_str in classification_results.items():
        classification_data = json.loads(json_str)
        species_paths = list(extract_paths(classification_data))
        # Purpose: Format the extracted classification paths and store them in the formatted_results dictionary.
        formatted_results = {}
        for species, path in species_paths:
            formatted_results[species] = {"Characteristics": path}
        # Purpose: Add the formatted classification results to final_results.
        final_results[key] = formatted_results
    # Purpose: Re-validate the corrected classification results and log any remaining errors.
    # Function: Call the validate_results function, passing in the updated final_results and knowledge_graph and returning a new list of errors. If there are no errors, then errors is empty and the loop ends.
    errors = validate_results(final_results, knowledge_graph)

# Save the final classification results
with open('final_classification.json', 'w') as f:
    json.dump(final_results, f, indent=4)

print("Final classification results have been saved to 'final_classification.json'.")
print(json.dumps(final_results, indent=4))

Final classification results have been saved to 'final_classification.json'.
{
    "1": {
        "Coccygidium": {
            "Characteristics": {
                "Character5": "1"
            }
        },
        "Biroia": {
            "Characteristics": {
                "Character5": "2",
                "Character6": "1"
            }
        },
        "Cremnoptoides": {
            "Characteristics": {
                "Character5": "2",
                "Character6": "1 and 2",
                "Character13": "1"
            }
        },
        "Disophrys": {
            "Characteristics": {
                "Character5": "2",
                "Character6": "1 and 2",
                "Character13": "2"
            }
        }
    },
    "2": {
        "Troticus": {
            "Characteristics": {
                "Character1": "2",
                "Character9": "2"
            }
        },
        "Hypsostypos": {
            "Characteristics": {
                "Character1": "2",

In [75]:
# Example initial result
initial_result = parsed_initial_classification

# Parse the API response JSON strings
parsed_classification_results = {key: json.loads(value) for key, value in classification_results.items()}

# Function to combine the initial and secondary classification results
def combine_results(initial, secondary, state_key):
    if not secondary:
        return

    initial_states = initial["States"].get(state_key)
    if initial_states is None:
        initial["States"][state_key] = secondary
        return

    if isinstance(initial_states, list):
        if isinstance(secondary, list):
            initial["States"][state_key] = list(set(initial_states + secondary))  # Merge two lists and remove duplicates
        else:
            initial["States"][state_key] = secondary
    elif isinstance(initial_states, dict):
        if isinstance(secondary, dict):
            for key, value in secondary["States"].items():
                if key not in initial_states:
                    initial_states[key] = value
                else:
                    combine_results(initial_states, value, key)
        else:
            raise ValueError(f"Conflicting types for key {state_key}: {type(initial_states)} vs {type(secondary)}")
    else:
        raise ValueError(f"Unexpected type for initial states: {type(initial_states)}")

# Dynamically combine all secondary classification results
for state_key, secondary in parsed_classification_results.items():
    combine_results(initial_result, secondary, state_key)

# Function to display the final classification result
def display_classification(result, indent=0):
    indent_space = " " * indent
    character = result.get("Character")
    states = result.get("States")

    classification = {}
    if character and states:
        classification["Character"] = character
        classification["States"] = {}
        print(f"{indent_space}1. **{character}:**")
        for state, species in states.items():
            if isinstance(species, list):
                print(f"{indent_space}   - State \"{state}\": {', '.join(species)}")
                classification["States"][state] = species
            elif isinstance(species, dict):
                print(f"{indent_space}   - State \"{state}\":")
                classification["States"][state] = display_classification(species, indent + 4)
    return classification

# Display the final classification result
final_result = display_classification(initial_result)
print("\nFinal Result JSON:")
# print(json.dumps(final_result, indent=2))


1. **Character1:**
   - State "1":
    1. **Character5:**
       - State "1": Coccygidium
       - State "2":
        1. **Character6:**
           - State "1": Biroia
           - State "1 and 2":
            1. **Character13:**
               - State "1": Cremnoptoides
               - State "2": Disophrys
   - State "2":
    1. **Character1:**
       - State "2":
        1. **Character9:**
           - State "2": Troticus
           - State "1": Hypsostypos
       - State "1 and 2": Cremnops
   - State "3": Gyrochus
   - State "4":
    1. **Character2:**
       - State "1":
        1. **Character8:**
           - State "1": Earinus
       - State "2":
        1. **Character4:**
           - State "2":
            1. **Character3:**
               - State "2": Therophilus
               - State "3": Bassus
           - State "1": Lytopylus
       - State "3":
        1. **Character8:**
           - State "3": Camptothlipsis, Aneurobracon

Final Result JSON:
